# 위상 정렬 (Topological Sorting)

## 줄 세우기

- 문제 출처: [백준 2252번](https://www.acmicpc.net/problem/2252)

`-` 위상 정렬 문제이다

`-` khan의 알고리즘 또는 DFS로 해결할 수 있다

`-` 여기서는 DFS로 해결하기로 함

`-` 나중에 봤을 때 이해가 안되면 알고리즘 수업 정리한거 찾아보자

`-` 참고로 모든 원소에 대해 비교한건 아니므로 한 번도 등장하지 않은 원소도 잊지말고 추가해야 한다 (어디든 상관없어서 마지막에 추가하면 된다)

In [2]:
import sys
from collections import defaultdict, deque

sys.setrecursionlimit(10**6)


def dfs(graph, node, stack):
    if node in visited:
        return
    visited[node] = True
    for next_ in graph[node]:
        if next_ not in visited:
            dfs(graph, next_, stack)
    stack.append(node)


def solution():
    global visited
    graph = defaultdict(list)
    visited = {}
    N, M = map(int, input().split())
    S = deque([])
    for _ in range(M):
        a, b = map(int, input().split())
        graph[a].append(b)
    for node in list(graph.keys()):
        stack = []
        dfs(graph, node, stack)
        S.extendleft(stack)
    for i in range(1, N + 1):
        if i not in visited:
            S.append(i)
    print(*S)


solution()

# input
# 4 2
# 4 2
# 3 1

 4 2
 4 2
 3 1


3 1 4 2
